In [137]:
import os
from collections import Counter
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold, train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing
from hyperopt import tpe

import neptune.new as neptune

ImportError: cannot import name '_ClassNamePrefixFeaturesOutMixin' from 'sklearn.base' (C:\Users\48502\miniconda3\envs\ml_project\lib\site-packages\sklearn\base.py)

In [138]:
import neptune.new as neptune

run = neptune.init(
    project="julia.grzegorowska/ml-project",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YzMwMDM2OC04YzdlLTQxOGEtYmEzYi0xZTA3ZmQzMjlkNzIifQ==",
)

params = {
    "optimizer": "Julia"
}
run["parameters"] = params


def send_data_neptune(data, plot_name):
    for epoch in range(0, len(data)):
      run[plot_name].log(data[epoch])

def single_record(record, record_name):
    run[record_name] = record

def stop_run():
    run.stop()

https://app.neptune.ai/julia.grzegorowska/ml-project/e/MLPROJ-8
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [139]:
os.environ["OMP_NUM_THREADS"] = "1"

In [140]:
seed = np.random.seed(147)

In [141]:
def data_load() -> list:

    train_data_out = pd.read_csv("data/train_data.csv", header=None)
    test_data_out = pd.read_csv("data/test_data.csv", header=None)
    train_labels_out = pd.read_csv("data/train_labels.csv", header=None)

    a = train_labels.values
    tmp = []
    for i in range(0, len(a)):
      tmp.append(int(a[i]))

    send_data_neptune(tmp, "train_labels")
    
    return [train_data, test_data, train_labels]

In [142]:
train_data, test_data, train_labels = data_load()

In [143]:
train_labels_ravel = train_labels.values.ravel()

In [171]:
def pipeline_standard_minmax(X_1, X_2: pd.DataFrame) -> np.array:

    pipeline = Pipeline([
    ("std", StandardScaler()),
    ("minmax", MinMaxScaler())])
    
    train_std_minmax = pipeline.fit_transform(X_1)
    test_std_minmax = pipeline.fit_transform(X_2)
    
    return [train_std_minmax, train_std_minmax]

In [172]:
train_std_minmax, test_std_minmax = pipeline_standard_minmax(train_data, test_data)

In [173]:
k=int(len(train_data.columns)/3)

In [174]:
def kbest_select(X_1, X_2, y_1: np.array, n_of_kbest: int) -> np.array:
    """Univariate Selection"""
    
    print(f"Shape before: {X_1.shape}\n")
    
    test = SelectKBest(score_func=f_classif, k=n_of_kbest)
    fit = test.fit(X_1, y_1)
    features_first = fit.transform(X_1)
    features_second = fit.transform(X_2)
    
    scores = fit.scores_
    score_df = pd.DataFrame(scores, columns=["Scores"])
    print(f"Min score: {min(score_df.Scores)}, max score: {max(score_df.Scores)}, mean score: {np.mean(score_df.Scores)}\n")    
    print(f"Shape after: {features_first.shape}\n")
    
    score_df.drop(score_df[score_df.Scores < 1].index, inplace=True)
    l = len(score_df)
    
    if l != n_of_kbest:
        return kbest_select(train_std_minmax, test_std_minmax, train_labels_ravel, l)
    else:
        return [features_first, features_second]

In [175]:
kbest_select, kbest_select = kbest_select(train_std_minmax, test_std_minmax, train_labels_ravel,k)

Shape before: (3750, 10000)

Min score: 1.7567042700994732e-09, max score: 17.321255892491074, mean score: 1.000380011727585

Shape after: (3750, 3333)

Shape before: (3750, 10000)

Min score: 1.7567042700994732e-09, max score: 17.321255892491074, mean score: 1.000380011727585

Shape after: (3750, 3177)



In [176]:
def pca_select(X_1, X_2: np.array) -> np.array:
    
    print(f"Shape before transforamtion: {X_1.shape}\n")
    
    scaler = MinMaxScaler(feature_range = (0, 1))
    pca = PCA(n_components=100, random_state=seed)
    fit = pca.fit(X_1)
    features_first = fit.transform(X_1)
    features_second = fit.transform(X_2)
    
    print(f"Explained Variance: \n{fit.explained_variance_ratio_}\n")
    print(f"Shape after transormation: {features_first.shape}")

    send_data_neptune(fit.explained_variance_ratio_)
    
    return [features_first, features_second]

In [178]:
pca_train, pca_test = pca_select(kbest_train, kbest_test)

NameError: name 'kbest_train' is not defined

In [179]:
def rfe_select(X_1, X_2, y_1: np.array) -> np.array:
    
    print(f"Shape before transformation: {X_1.shape}\n")
    
    model = LogisticRegression()
    svc = SVC(kernel="linear", C=1, random_state=seed)
    rfe = RFE(estimator=svc, n_features_to_select=5)
    fit = rfe.fit(X_1, y_1)
    first_features = fit.transform(X_1)
    second_features = fit.transform(X_2)
    
    print(f"Feature Ranking: \n{fit.ranking_}\n")
    print(f"Shape after: {first_features.shape}\n")

    send_data_neptune(fit.ranking_)
    
    return [first_features, second_features]

In [180]:
rfe_train, rfe_test = rfe_select(pca_train, pca_test, train_labels_ravel)

Shape before transformation: (3750, 100)

Feature Ranking: 
[ 1 78  4 17 84  2 85 94 36  1 38 33 75 10 81 73 11 64 42 44  3  7 70 52
 69 20 40  9 74 95 83 30 56 13 32 57 65 76 79 27 62 66 47  1  5 26  1 92
  8 41 34 25 45 58 22 12 50 90 35 60 80 29 71 59 53 67 82 46 14  6 89 31
 23 55 51 61 16 96 88 24 91 86 28 15 48 19 93 18 63 77 43 49  1 72 87 54
 37 68 39 21]

Shape after: (3750, 5)



TypeError: send_data_neptune() missing 1 required positional argument: 'plot_name'